In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import typing

In [20]:
anio = 2018

In [21]:
data = pd.read_csv(f'/home/jovyan/work/data/raw/{anio}.csv')

In [22]:
# This wrapper will apply any sklearn transform you pass into it to a group.
class SklearnWrapper:
    def __init__(self, transformation: typing.Callable):
        self.transformation = transformation

    def __call__(self, df):
        transformed = self.transformation.fit_transform(df.values)
        return pd.DataFrame(transformed, columns=df.columns, index=df.index)

In [23]:
grouped = data.groupby(['ORIGIN','FL_DATE']).agg({'DEP_DELAY': 'mean'})

df_rescaled = (
    grouped.groupby("ORIGIN")
    .apply(SklearnWrapper(StandardScaler()))
    .rename(columns = {'DEP_DELAY': 'SCALED'})
)
df_rescaled = (
    grouped.join(df_rescaled)
    .assign(OUTLIER=(abs(df_rescaled.SCALED)>=3).astype(int))
)

In [24]:
df_rescaled.head()

DEP_DELAY    SCALED  OUTLIER
ORIGIN FL_DATE                                 
ABE    2018-01-01   7.833333 -0.203821        0
       2018-01-02  77.375000  2.849058        0
       2018-01-03  51.333333  1.705829        0
       2018-01-04  30.125000  0.774783        0
       2018-01-05  17.375000  0.215058        0

In [25]:
df_rescaled.OUTLIER.sum()

2315